<a href="https://github.com/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_1_CMRSET_Long_Term_Average_ET_(Temporal_Aggregation).ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View on GitHub"/></a>
&nbsp;
<a href="https://colab.research.google.com/github/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_1_CMRSET_Long_Term_Average_ET_(Temporal_Aggregation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
&nbsp;
<a href="https://mybinder.org/v2/gh/ternaustralia/cmrset-examples/HEAD?labpath=gee-examples%2Fpython%2FExample_1_CMRSET_Long_Term_Average_ET_(Temporal_Aggregation).ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>


# Example 1 - CMRSET Long-Term Average ET (Temporal Aggregation)
This example illustrates how to access the CMRSET Landsat V2.2 data product via Earth Engine and compute the long-term average evapotranspiration for a section of the Lachlan River.  Finally the result is exported to Google Drive as a Cloud-Optimised GeoTIFF.

To run this example it is assumed you have [registered](https://signup.earthengine.google.com/) to [Google Earth Engine](https://earthengine.google.com/). Please feel free to adapt this example to meet your own requirements.

---
**Execution Time:** ~20 minutes

**Storage Required:** 55 MB (on Google Drive)

## Install earthengine-api


In [ ]:
# Installs the geemap package
try:
    import ee
except ImportError:
    print('Installing earth engine...')
    !python -m pip install earthengine-api

## Authenticate and initialize
Run the `ee.Authenticate()` function to authenticate your access to Earth Engine servers and `ee.Initialize()` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. **Please follow the instructions in the prompts.** You will also need to enter your Google Earth Engine Cloud Project ID below.

In [ ]:
# Import the Earth Engine module
import ee

# Cloud Project ID to associate with your Earth Engine workspace.
project = "Your Project ID to use"

try:
    ee.Initialize(project=project) # Initialize the library.
except Exception as e:
    ee.Authenticate() # If initialization fails, authentication is required... Trigger the authentication flow and initialize again.
    ee.Initialize(project=project) # Initialize the library.

## Compute the long-term ET average
Firstly, define the extents of your region of interest. We will do this by specifying the extents of our rectangle.

In [ ]:
# Define the region of interest.
region = ee.Geometry.Rectangle([145.0, -34.0, 146.0, -33.0])

Next, we will access [CMRSET](https://developers.google.com/earth-engine/datasets/catalog/TERN_AET_CMRSET_LANDSAT_V2_2) from the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets), and filter down the [ImageCollection](https://developers.google.com/earth-engine/tutorials/tutorial_api_04) to the period of interest.

In [ ]:
# Define the period of interest.
start = '2000-02-01'
end = '2021-03-01'

# Access evapotranspiration from the CMRSET dataset.
cmrset_aet = ee.ImageCollection("TERN/AET/CMRSET_LANDSAT_V2_2").filterDate(start, end).select(['ETa'])

Lastly, we will compute the long-term average of all images in the filtered ImageCollection using a mean [reducer](https://developers.google.com/earth-engine/guides/reducers_image_collection). We will do this using the shortcut function `mean()`. We will also cast the datatype of the result from [*double precision*](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) to [*single precision*](https://en.wikipedia.org/wiki/Single-precision_floating-point_format) using the `toFloat()` function.  This will minimise the file size of the export, since double precision is unnecessary for evapotranspiration values.

In [ ]:
# Calculate long-term average map over specified time period.
mean_aet = cmrset_aet.mean().toFloat()

## Preview the result
Do you know why all Irish whiskey is triple distilled? To be sure, to be sure, to be sure... Now let's make sure the data is exactly what we want to export.  We will do this by previewing the image as a [thumbnail](https://developers.google.com/earth-engine/guides/image_visualization#thumbnail-images).

In [ ]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of the CMRSET long-term average.
Image(url = mean_aet.getThumbURL({'min': 0,
                                  'max': 7,
                                  'region': region,
                                  'dimensions': 512,
                                  'palette': ["d7191c","fdae61","ffffbf","abd9e9","2c7bb6"]
                                  }))

## Export the image to Google Drive
Define an [export task](https://developers.google.com/earth-engine/guides/exporting) from Earth Engine which will [export the image to Google Drive](https://developers.google.com/earth-engine/apidocs/export-image-todrive) as a [Cloud-Optimised GeoTIFF](https://www.cogeo.org/) (COG).

In [ ]:
# Define the export settings.
scale = 30
export_folder = 'CMRSET_Exports'
task_description = 'CMRSET_{0}_{1}_{2}m'.format(start, end, scale)

# Export the image, specifying the region and scale.
task = ee.batch.Export.image.toDrive(**{
    'image': mean_aet,
    'region': region,
    'scale': scale,
    'description': task_description,
    'folder': export_folder
})
task.start()

Now, we will continue monitoring the task until it is complete. Each time the task status changes state, it will be printed to the standard output below. When the task has completed, you will find the file in [your Google Drive account](https://drive.google.com/).

In [ ]:
import time

# Function to report the status to the standard output.
status = {'state': None}
def reportStatus():
  currentStatus = task.status()
  if currentStatus['state'] != status['state']:
    print('[Task:{0}] - State changed to {1}'.format(task.id, currentStatus['state']))
  return currentStatus

# Continue to monitor the task status when it changes.
while task.active():
  status = reportStatus()
  time.sleep(5)

# Output any messages if the task is no longer active.
status = reportStatus()
if 'destination_uris' in status: # Task completed
  print('[Task:{0}] - Data location: {1}'.format(task.id, status['destination_uris']))
if 'error_message' in status: # Task failed
  print('[Task:{0}] - Failed with error: {1}'.format(task.id, status['error_message']))

[Task:LWHTQYIWELNI75D5FIFRDGW6] - State changed to READY
[Task:LWHTQYIWELNI75D5FIFRDGW6] - State changed to RUNNING


You may also monitor the task progress under the **Tasks** tab in the [Earth Engine Code Editor](https://code.earthengine.google.com/) or via the [Earth Engine Task Manager](https://code.earthengine.google.com/tasks)